In [1]:
# !pip install openai
# !pip install socksio

In [1]:
from openai import OpenAI

In [20]:
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

system_prompt = """您是一位具备高级推理能力的专家AI助手。您的任务是为您的思考过程提供详细、逐步的解释。对于每个步骤：
1. 提供一个清晰、简洁的标题描述当前的推理阶段。
2. 在内容部分详细阐述您的思考过程。
3. 决定是继续推理还是提供最终答案。
响应格式：
使用JSON，包含以下键：title（标题），content（内容），next_action（取值：continue 或 final_answer）

关键要求：
- 采用至少5个明确的推理步骤。
- 承认您作为AI的局限性，并明确说明您能做什么以及不能做什么。
- 主动探索和评估可能的替代答案或方法。
- 批判性地评估您的推理过程；识别潜在的缺陷或偏见。
- 重新检查时，使用完全不同的方法或视角。
 -采用至少3种不同的方法来推导或验证答案。
- 在推理中结合相关领域知识和最佳实践。
- 在可能的情况下，为每个步骤和最终结论量化确定性水平。
- 考虑潜在的边界情况或例外情况。
- 提供清晰的理由来排除其他假设。

有效JSON响应的示例：
{
    "title": "初步问题分析",
    "content": "为了有效解决这个问题，我将首先把给定的信息分解为关键组成部分。这包括识别...[详细解释]...通过这种方式结构化问题，我们可以系统地解决每个方面。",
    "next_action": "continue"
}


"""


system_prompt = """您是一位具备高级推理能力、语义解析和数据库的专家AI助手。您的任务是为用户问题拆分为多个子问题，根据多个子问题的答案回答原始的用户问题。具体要求：
1. 对于用户问题，一步一步地编写具体的解答过程，放入到<thought></thought>标签。
2. 根据原始问题和解答过程，将原始问题拆分为独立的子问题，具体要求：
   - 每个子问题要求简单、详细和完整
   - 每个子问题之间尽量保持连续性
   - 如果上一个子问题的答案是下一个子问题的输入，那么答案以<response_数字>标志放入到下一个子问题中，并在每个子问题中包含该答案标志<response_数字>，数字以1开始，逐渐增加。
   - 每个子问题都需要json，包含以下键：subquery（拆分后的子问题），answer(答案标志<response_数字>), next_action（取值：continue 或 final_answer）
   - 所有子问题的json组成列表

完整的样例：
用户问题：在耗电量最大的设备是哪一年制造的？
子问题列表：
<subquery>

{
    "subquery": "查询耗电量最大的设备是什么",
    "answer": "response_1"
    "next_action": "continue"
}
{
    "subquery": "<response_1>是哪一年制造的",
    "answer": "response_2"
    "next_action": "finished"
}
</subquery>


"""




In [ ]:

result = {"thought": "为了回答这个问题，我们需要先确定功率最大的发电机是什么，然后找出这个发电机的购买时间。"}



In [21]:
chat_response = client.chat.completions.create(
    model="Qwen2.5-7B-Instruct",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "功率最大的发电机和是什么时候购买的"},
    ],
    temperature=0.001,
    # top_p=0.8,
    max_tokens=1024,
    extra_body={
        "repetition_penalty": 1.05,
    },
)
# print("Chat response:", chat_response)
print(chat_response.choices[0].message.content)



<thought>
为了回答这个问题，我们需要先确定功率最大的发电机是什么，然后找出这个发电机的购买时间。
</thought>
子问题列表：
<subquery>
{
    "subquery": "查询功率最大的发电机是什么",
    "answer": "response_1",
    "next_action": "continue"
}
{
    "subquery": "response_1 是什么时候购买的",
    "answer": "response_2",
    "next_action": "final_answer"
}
</subquery>


In [22]:
chat_response = client.chat.completions.create(
    model="Qwen2.5-7B-Instruct",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "功率最大的发电机和耗电量最大的设备，谁买的比较早"},
    ],
    temperature=0.001,
    # top_p=0.8,
    max_tokens=1024,
    extra_body={
        "repetition_penalty": 1.05,
    },
)
# print("Chat response:", chat_response)
print(chat_response.choices[0].message.content)



<thought>
为了回答这个问题，我们需要先确定功率最大的发电机是哪一年制造的，然后确定耗电量最大的设备是哪一年制造的，最后比较这两个日期来判断哪个更早。
</thought>
子问题列表：
<subquery>
{
    "subquery": "查询功率最大的发电机是哪一年制造的",
    "answer": "response_1",
    "next_action": "continue"
}
{
    "subquery": "查询耗电量最大的设备是哪一年制造的",
    "answer": "response_2",
    "next_action": "continue"
}
{
    "subquery": "<response_1>和<response_2>哪个日期更早",
    "answer": "response_3",
    "next_action": "final_answer"
}
</subquery>


In [1]:
"""指导vLLM结构化输出"""

from pydantic import BaseModel, Field
from typing import Optional, List, Annotated
from enum import Enum


class NextAction(str, Enum):
    continuing = "continuing"
    finished = "finished"



class SubQuery(BaseModel):

    subquery: Annotated[str, Field(description="子问题")]
    answer: Annotated[str, Field(description="子问题的答案标志<response_数字>")]
    next_action: Annotated[NextAction, Field(description="下一步动作是继续continue还是返回答案final_answer")]


class QueryDecomposition(BaseModel):

    subquery_list: Annotated[List[SubQuery], Field(description="原始问题拆解后形成的子问题列表")]
    



In [2]:

chat_response = client.chat.completions.create(
    model="Qwen2.5-7B-Instruct",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "功率最大的发电机和耗电量最大的设备，谁买的比较早"},
    ],
    temperature=0.001,
    # top_p=0.8,
    max_tokens=1024,
    extra_body={
        "repetition_penalty": 1.05,
        "guide_json": QueryDecomposition.model_json_schema()
    },
    # extra_body={"guide_json": QueryDecomposition.model_json_schema()}
)
# print("Chat response:", chat_response)
print(chat_response.choices[0].message.content)



NameError: name 'client' is not defined

In [29]:
print(chat_response)

ChatCompletion(id='chatcmpl-a904f7fa63b841469c3e4f88627b8114', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<thought>\n为了回答这个问题，我们需要先确定功率最大的发电机是哪一年制造的，然后确定耗电量最大的设备是哪一年制造的，最后比较这两个日期来得出哪个设备购买得更早。\n</thought>\n子问题列表：\n<subquery>\n{\n    "subquery": "查询功率最大的发电机是哪一年制造的",\n    "answer": "response_1",\n    "next_action": "continue"\n}\n{\n    "subquery": "查询耗电量最大的设备是哪一年制造的",\n    "answer": "response_2",\n    "next_action": "continue"\n}\n{\n    "subquery": "<response_1>和<response_2>哪个日期更早",\n    "answer": "response_3",\n    "next_action": "final_answer"\n}\n</subquery>', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[]), stop_reason=None)], created=1735370109, model='Qwen2.5-7B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=172, prompt_tokens=353, total_tokens=525, completion_tokens_details=None, prompt_tokens_details=None), prompt_logp

In [36]:
import re
pattern = "<subquery>(.*?)</subquery>"

# 提取所有匹配内容
matches = re.findall(pattern, chat_response.choices[0].message.content)

for match in matches:
    print(match)


<>:2: SyntaxWarning: invalid escape sequence '\<'
<>:2: SyntaxWarning: invalid escape sequence '\<'
/tmp/ipykernel_5039/4289170415.py:2: SyntaxWarning: invalid escape sequence '\<'
  pattern = "\<subquery\>(.*?)\</subquery\>"


In [37]:
text = chat_response.choices[0].message.content
print(type(text), text)
print(re.match(pattern, text))

<class 'str'> <thought>
为了回答这个问题，我们需要先确定功率最大的发电机是哪一年制造的，然后确定耗电量最大的设备是哪一年制造的，最后比较这两个日期来得出哪个设备购买得更早。
</thought>
子问题列表：
<subquery>
{
    "subquery": "查询功率最大的发电机是哪一年制造的",
    "answer": "response_1",
    "next_action": "continue"
}
{
    "subquery": "查询耗电量最大的设备是哪一年制造的",
    "answer": "response_2",
    "next_action": "continue"
}
{
    "subquery": "<response_1>和<response_2>哪个日期更早",
    "answer": "response_3",
    "next_action": "final_answer"
}
</subquery>
None
